In [1]:
import pandas as pd
# Set max columns to display
pd.set_option('display.max_columns', None)

import numpy as np
from sklearn import preprocessing as pre
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



C:\Users\austinsh\AppData\Local\Temp\ipykernel_41788\1304120184.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# # Importing CSV files
# df_CDunit = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_554Data_clean.csv')
# df_AlCon = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_425Data_clean.csv')
# df_FB554 = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_unitData_clean.csv')


In [3]:
# # Importing CSV files
# bordeCode directory
df_CDunit = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Continuous Data\cont_unitData_clean.csv')
df_AlCon = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Continuous Data\cont_425Data_clean.csv')
df_FB554 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Continuous Data\cont_554Data_clean.csv')


In [4]:
print(df_CDunit.describe())
print(df_AlCon.describe())
print(df_FB554.describe())


            FC55003       FC55009       FC55552       FC55569      FFC55553  \
count  52974.000000  52974.000000  52974.000000  52974.000000  52974.000000   
mean    5971.598388    838.409551  35721.025258   6589.922506      1.001760   
std      870.091519    612.978542   5287.506602    391.888390      0.038444   
min     2730.450000      0.000000  17775.100000   5119.710000      0.809025   
25%     5446.792500    282.864750  34563.250000   6348.302500      0.977791   
50%     5992.205000    822.585500  37944.800000   6572.995000      0.998927   
75%     6545.882500   1318.387500  39013.300000   6807.497500      1.020570   
max     9207.210000   2674.790000  52000.000000   8126.760000      1.184630   

           FFC55555       LC90366       LC90368       PI55020       TC55552  \
count  52974.000000  52974.000000  52974.000000  52974.000000  52974.000000   
mean       0.771663     46.213319     36.769280     -1.463556    168.919046   
std        0.023297     28.995961     20.170483    

In [5]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: object
Data type for 'Date' column in df_FB554: object
Data type for 'Date' column in df_AlCon: object


In [6]:
df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')
df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')


In [7]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [8]:
print(df_CDunit.columns)
print(df_FB554.columns)
print(df_AlCon.columns)


Index(['Date', 'FC55003', 'FC55009', 'FC55552', 'FC55569', 'FFC55553',
       'FFC55555', 'LC90366', 'LC90368', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')
Index(['Date', 'Decanol'], dtype='object')
Index(['Date', '425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O'],
      dtype='object')


In [9]:
def apply_rolling_average_to_df(df, rolling_size):
    # Ensure 'Date' is the index if it's not already
    if df.index.name != 'Date':
        df = df.set_index('Date')

    # Apply rolling average to all columns
    rolled_df = df.rolling(window=rolling_size, min_periods=1).mean()

    # Reset index to make 'Date' a column again
    rolled_df = rolled_df.reset_index()

    return rolled_df



In [10]:
def apply_time_shift_by_hours(df, shift_hours):
    """
    Shifts the DataFrame's datetime index by the specified number of hours.

    :param df: DataFrame with 'Date' as its datetime index or column.
    :param shift_hours: Number of hours to shift. Can be positive (forward) or negative (backward).
    :return: Shifted DataFrame.
    """
    # Convert 'Date' to datetime and set as index if it's not already
    if df.index.name != 'Date':
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.set_index('Date')

    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)

    # Shift the DataFrame's index by the specified number of hours
    df.index = df.index + pd.Timedelta(hours=shift_hours)

    return df

In [11]:
# # Usage Examples
# shift_hours_AlCon = 1  # Negative shift for df_AlCon (e.g., -5 hours backward)
# shift_hours_FB554 = 5   # Positive shift for df_FB554 (e.g., 5 hours forward)

# shifted_df_AlCon = apply_time_shift_by_hours(df_AlCon, shift_hours_AlCon)
# print("Shifted df_AlCon:")
# print(shifted_df_AlCon.head())

# shifted_df_FB554 = apply_time_shift_by_hours(df_FB554, shift_hours_FB554)
# print("\nShifted df_FB554:")
# print(shifted_df_FB554.head())

In [12]:
def join_df_FB554_to_df_CDunit(df_CDunit, df_FB554):
    # Reset index if 'Date' is the index
    if df_CDunit.index.name == 'Date':
        df_CDunit = df_CDunit.reset_index()
    if df_FB554.index.name == 'Date':
        df_FB554 = df_FB554.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
    df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')

    df_CDunit = df_CDunit.dropna(subset=['Date']).sort_values('Date')
    df_FB554 = df_FB554.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df = pd.merge_asof(df_FB554, df_CDunit, on='Date', direction='nearest')

    return combined_df

def join_df_AlCon_to_combined_df(combined_df, df_AlCon):
    # Reset index if 'Date' is the index
    if combined_df.index.name == 'Date':
        combined_df = combined_df.reset_index()
    if df_AlCon.index.name == 'Date':
        df_AlCon = df_AlCon.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')

    combined_df = combined_df.dropna(subset=['Date']).sort_values('Date')
    df_AlCon = df_AlCon.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df_all = pd.merge_asof(df_AlCon, combined_df, on='Date', direction='nearest')
    
    return combined_df_all


In [13]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)



Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [14]:
def process_data_limited():
    # Apply the specific rolling average directly
    rolled_df_CDunit = apply_rolling_average_to_df(df_CDunit, 8)
    rolled_df_FB554 = apply_rolling_average_to_df(df_FB554, 4)
    rolled_df_AlCon = apply_rolling_average_to_df(df_AlCon, 2)

    # Apply the specific time shifts directly
    rolled_df_AlCon_shifted = apply_time_shift_by_hours(rolled_df_AlCon, -1) # Assuming apply_time_shift_by_hours handles negative shifts correctly
    rolled_df_FB554_shifted = apply_time_shift_by_hours(rolled_df_FB554, 1)

    # Combine df_CDunit and df_FB554 to create combined_df
    combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554_shifted)

    # Combine combined_df with rolled_df_AlCon to create combined_df_all
    combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon_shifted)

    # At this point, combined_df_all is the DataFrame with the data processed by the specified shifts and averages
    return combined_df_all


In [15]:
# Make sure all your helper functions and initial DataFrames (df_CDunit, df_FB554, df_AlCon) are correctly defined

# Now, call the modified process_data function to get the processed DataFrame
final_dataset = process_data_limited()

# Inspect the final_dataset
print(final_dataset.head())  # Print the first few rows to inspect the dataset


                 Date  425_pct_Al   M_Value  C4_pct_Eth  C4_pct_H2O  \
0 2012-05-14 07:00:00     6.26763  3.551350    2.353670    21.66280   
1 2012-05-14 08:00:00     6.26798  3.551080    2.355025    21.66545   
2 2012-05-14 09:00:00     6.26868  3.550545    2.357740    21.67070   
3 2012-05-14 10:00:00     6.26938  3.550010    2.360455    21.67595   
4 2012-05-14 11:00:00     6.27008  3.549475    2.363170    21.68125   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O   Decanol      FC55003  \
0            0.519068    0.620812     1.088680  0.959336  6332.030000   
1            0.553096    0.621150     1.084100  0.959336  6332.030000   
2            0.621152    0.621824     1.074945  0.959336  6332.030000   
3            0.667952    0.622498     1.065790  0.933602  6191.103333   
4            0.693497    0.623173     1.056635  0.907868  6081.507500   

     FC55009       FC55552      FC55569  FFC55553  FFC55555    LC90366  \
0  2512.3700  41080.000000  6432.660000  1.000685  0.749378 

In [16]:
final_dataset.columns

Index(['Date', '425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Decanol', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')

In [17]:
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# model_results.to_csv('merged_data'.csv', index=False)
                     
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# df_CD.to_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\1 Preprocess\Continuous Data\contData_all.csv', index=False)            

final_dataset.to_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Merge Data\merged_data.csv', index=False)


In [18]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-05-06 01:03:20.346079
